# OpenAI Agent with LlamaIndex

## Install Dependencies

In [4]:
# !source /Users/joshjung/Library/Mobile\ Documents/com~apple~CloudDocs/Documents/personalProjects/aiagenthackathon/venv/bin/activate
!echo "YOUR API KEY" > .env
# !which python3



In [5]:
# %pip install uv
# %uv pip install --system -qU llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv
# !which python3

## Setup API Keys


In [6]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Import libraries and setup LlamaIndex

In [7]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4")

# Set up Phoenix

In [10]:
# import phoenix as px
# session = px.launch_app()

In [12]:
# from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
# from phoenix.otel import register

# tracer_provider = register()
# LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

## Load Documents

In [26]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/asylum"
    )
    asylum_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/niw"
    )
    niw_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

This is the point we create our vector indexes, by calculating the embedding vectors for each of the chunks. You only need to run this once.

In [30]:
if not index_loaded:
    # load data
    asylum_docs = SimpleDirectoryReader(
        # input_files=["./docs/asylum/How to Seek Asylum in the United States.pdf","./docs/asylum/i-589instr.pdf","./docs/asylum/Immigration Law Sourcebook.pdf"]
        input_dir = "./docs/asylum",
        recursive=True
    ).load_data()
    niw_docs = SimpleDirectoryReader(
        # input_files=["./docs/niw/.pdf"]
        input_dir="./docs/niw/", recursive=True
    ).load_data()

    # build index
    asylum_index = VectorStoreIndex.from_documents(asylum_docs, show_progress=True)
    niw_index = VectorStoreIndex.from_documents(niw_docs, show_progress=True)

    # persist index
    asylum_index.storage_context.persist(persist_dir="./storage/asylum")
    niw_index.storage_context.persist(persist_dir="./storage/niw")

Parsing nodes:   0%|          | 0/2745 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1643 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/171 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/182 [00:00<?, ?it/s]

Now create the query engines.

In [31]:
asylum_engine = asylum_index.as_query_engine(similarity_top_k=3, llm=llm)
niw_engine = niw_index.as_query_engine(similarity_top_k=3, llm=llm)

We can now define the query engines as tools that will be used by the agent.

As there is a query engine per document we need to also define one tool for each of them.

In [32]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=asylum_engine,
        metadata=ToolMetadata(
            name="asylum_docs",
            description=(
                "Provides information about asylum application process in the United States. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=niw_engine,
        metadata=ToolMetadata(
            name="niw_docs",
            description=(
                "Provides information about NIW green card application process. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Creating the Agent
Now we have all the elements to create a LlamaIndex ReactAgent

In [35]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

Now we can interact with the agent and ask a question.

In [18]:
response = agent.chat("I am starting the asylum process. What do I need to do as the first step?")
print(str(response))


> Running step cd04bb65-251a-482d-b3f1-a5d8a131deec. Step input: I am starting the asylum process. What do I need to do as the first step?
Thought: The user is asking about the first step in the asylum process in the United States. I can use the asylum_docs tool to get this information.
Action: asylum_docs
Action Input: {'input': 'What is the first step in the asylum process?'}
Observation: The first step in the asylum process is to file affirmatively for asylum. After this, you will have to wait for weeks, months or even years for an interview.
> Running step eccf7a33-85af-466b-ad2b-907d2d71a466. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The first step in the asylum process is to file affirmatively for asylum. After filing, you may have to wait for weeks, months, or even years for an interview.
The first step in the asylum process is to file affirmatively for asylum. After filing, you may have to wait for weeks

In [25]:
user_input = """ 
I crossed the southern border of the United States on 2025-06-19. I am a citizen of the Republic ofSouth Africa. I am applying for asylum. I have a passport and address in the US
"""
response = agent.chat(f"""Answer as detail as possible based on below context.

                      Tone: Inquisitive and insightful, with a gentle sense of respect for the subject matter.

                      Situation: {user_input}

                      Emphasis: Use specific form number and descriptive language so the user can fill out all the necessary information without help from an attorney.

                      Example: You need the form I-589, your passport number, specific link to each document.

                      Format: Use number list or bullet list.

                      If the context does not contain the required information, search the web.
""")
print(str(response))

> Running step f1e011a4-8844-4d61-8a94-a7d245166635. Step input: Answer as detail as possible based on below context.

                      Tone: Inquisitive and insightful, with a gentle sense of respect for the subject matter.

                      Situation:  
I crossed the southern border of the United States on 2025-06-19. I am a citizen of the Republic ofSouth Africa. I am applying for asylum. I have a passport and address in the US


                      Emphasis: Use specific form number and descriptive language so the user can fill out all the necessary information without help from an attorney.

                      Example: You need the form I-589, your passport number, specific link to each document.

                      Format: Use number list or bullet list.

                      If the context does not contain the required information, search the web.

Thought: The user is asking for a detailed guide on how to apply for asylum in the United States, including the s

In [36]:
user_input = """ 
I did a PhD at University of Washington in Electrical Engineering and wrote 5 papers with citations more than 100. I want to start the NIW green card process.
"""
response = agent.chat(f"""Answer as detail as possible based on info below.

                      Tone: Inquisitive and insightful, with a gentle sense of respect for the subject matter.

                      Situation: {user_input}

                      Emphasis: Use specific form number and descriptive language so the user can fill out all the necessary information without help from an attorney.

                      Example: You need the form I-589, your passport number, you need forms and documents from specific links. Give examples of proposed endeavors for NIW cases.

                      Format: Use number list or bullet list.

                      Also, include a checklist for the whole documents.

                      If the context does not contain the required information, search the web. Plus, based on what step is the user is at, provide prospective steps and froms the user needs in detail. 

                      Exception: If user asks a specific question regarding a form, answer it briefly. For example, if the user asks what he/she should put as the passport number if he/she has two passports, first try to answer based on the context, if the answer not found, answer based on general practices. 
""")
print(str(response))

> Running step 9ff57023-6273-43dd-82fb-c1fe6623a7bb. Step input: Answer as detail as possible based on info below.

                      Tone: Inquisitive and insightful, with a gentle sense of respect for the subject matter.

                      Situation:  
I did a PhD at University of Washington in Electrical Engineering and wrote 5 papers with citations more than 100. I want to start the NIW green card process.


                      Emphasis: Use specific form number and descriptive language so the user can fill out all the necessary information without help from an attorney.

                      Example: You need the form I-589, your passport number, you need forms and documents from specific links. Give examples of proposed endeavors for NIW cases.

                      Format: Use number list or bullet list.

                      Also, include a checklist for the whole documents.

                      If the context does not contain the required information, search the

In [38]:
user_input = """ 
I am going through the NIW process. I got recomeendation letters from 3 collaborators. I am filling out I-140. What passport number should I put if I have dual citizenship from Canada and China. I am doing my process in Canada.
"""
response = agent.chat(f"""Answer as detail as possible based on info below.

                      Tone: Inquisitive and insightful, with a gentle sense of respect for the subject matter.

                      Situation: {user_input}

                      Emphasis: Use specific form number and descriptive language so the user can fill out all the necessary information without help from an attorney.

                      Example: You need the form I-589, your passport number, you need forms and documents from specific links. Give examples of proposed endeavors for NIW cases.

                      Format: Use number list or bullet list.

                      Also, include a checklist for the whole documents.

                      If the context does not contain the required information, search the web. Plus, based on what step is the user is at, provide prospective steps and froms the user needs in detail. 

                      Exception: If user asks a specific question regarding a form, answer it briefly. For example, if the user asks what he/she should put as the passport number if he/she has two passports, first try to answer based on the context, if the answer not found, answer based on general practices. 
""")
print(str(response))

> Running step 60dc8bda-07ca-4550-ad81-eb9953378789. Step input: Answer as detail as possible based on info below.

                      Tone: Inquisitive and insightful, with a gentle sense of respect for the subject matter.

                      Situation:  
I am going through the NIW process. I got recomeendation letters from 3 collaborators. I am filling out I-140. What passport number should I put if I have dual citizenship from Canada and China. I am doing my process in Canada.


                      Emphasis: Use specific form number and descriptive language so the user can fill out all the necessary information without help from an attorney.

                      Example: You need the form I-589, your passport number, you need forms and documents from specific links. Give examples of proposed endeavors for NIW cases.

                      Format: Use number list or bullet list.

                      Also, include a checklist for the whole documents.

                     